# Rescore docking poses with predicted binding affinity

Input is
1. docking poses with rank and RMSD
2. binding affinities predicted for each pose and its receptor
3. binding affinities predicted for each reference ligand and its receptor

With the predicted affinity, we can re-rank the docking poses. To evaluate, if this brings any improvement for a specific docking, we compare the RMSD of the pose ranked highest by the docking program, and the pose with the highest affinity. If the top pose based on affinity has a lower RMSD, the re-ranking did improve the docking result.

To evaluate the rescoring based on the entire set of dockings, we count the number of top poses with an RMSD below several thresholds. We compare these counts before and after rescoring.

To directly compare rescoring with the scoring function used by the docking program, we count dockings where the best pose (RMSD-wise) is also the top ranked pose.

The predicted binding affinity for reference ligands opens another way of evaluating the rescoring: if it works well, the reference ligands should be assigned an affinity higher than all their docking poses.

## Load and prepare RMSD and affinity data

In [1]:
# SELECT DATA TO LOAD

# rf or gbt
model = 'gbt'
ecif_cutoff = 8.5

In [2]:
# UTILITIES

import math

def df_to_latex(df, ncol=1):
    """
    Prints a DataFrame as LaTeX table. When ncol>1, the rows are grouped into chunks and
    pasted together horizontally.
    
    An example for ncol=2:
    
    The input
    
       a  b
    0  1  4
    1  2  5
    2  3  6
    
    becomes
    
    1	&	4	&	3	&	6 \\
    2	&	5 \\
    """
    # reset index so that integer indexing works as expected
    df = df.reset_index(drop=True)
    if ncol == 1:
        for _, row in df.iterrows():
            print('\t&\t'.join(str(item) for item in row), end=' \\\\\n')
        return
    
    # split input frame into one frame per column, so that the first
    # ranges from row 0 to math.ceil(len(df) / ncol), the second
    # contains the same number of following rows, etc, until all rows
    # are consumed.
    n_rows_per_col = math.ceil(len(df) / ncol)
    start_idx = 0
    frames = []
    while len(frames) < ncol:
        # -1 because end index is inclusive in pandas
        end_idx = start_idx + n_rows_per_col - 1
        # reset index so that integer indexing works as expected
        frames.append(df.loc[start_idx:end_idx, ].reset_index(drop=True))
        start_idx += n_rows_per_col
    # the ith output row contains the ith row from each data frame pasted
    # together and formatted as LaTeX table
    for i in range(len(frames[0])):
        print('\t&\t'.join(str(item) for frame in frames if len(frame) > i for item in frame.loc[i, ]), end=' \\\\\n')

In [3]:
import pandas as pd

docking = pd.read_csv(
    f'/home/Luis/Repos/Uni/Masterarbeit/ma-results/LEADS-FRAG/docking/glide-dock/8/rmsd_all_poses.csv')
predictions = pd.read_csv(
    f'/home/Luis/Repos/Uni/Masterarbeit/ma-results/LEADS-FRAG/rescoring/binding_affinity_prediction/docking_8/{model}/ECIF_LD_{ecif_cutoff}-{model.upper()}.csv')
refligs = pd.read_csv(
    f'/home/Luis/Repos/Uni/Masterarbeit/ma-results/LEADS-FRAG/rescoring/binding_affinity_prediction/ref-ligands/{model}/ECIF_LD_{ecif_cutoff}-{model.upper()}.csv')

The data frames look as follows:

In [4]:
docking

,Receptor,PoseRank,RMSD
0,1Q11.A,1,10.889195
1,1Q11.A,2,11.078528
2,1Q11.A,3,11.170565
3,1Q11.A,4,10.772341
4,1Q11.A,5,10.793099
...,...,...,...
284,5ILW.A,1,3.631839
285,5ILW.A,2,0.587167
286,5ILW.A,3,3.868636
287,5ILW.A,4,3.838628


In [5]:
predictions

,Receptor,Ligand,Predicted_Binding_Affinity
0,1Q11,1,4.336701
1,1Q11,2,4.221958
2,1Q11,3,4.097513
3,1Q11,4,4.077734
4,1Q11,5,4.081782
...,...,...,...
305,5ILW,1,4.862881
306,5ILW,2,5.091005
307,5ILW,3,4.811201
308,5ILW,4,4.795057


To make life easier, make receptor IDs and column names match between both frames.

In [6]:
docking.Receptor = [pdb.split('.')[0] for pdb in docking.Receptor]
predictions.rename(columns={'Ligand': 'PoseRank'}, inplace=True)

In [7]:
docking.head()

,Receptor,PoseRank,RMSD
0,1Q11,1,10.889195
1,1Q11,2,11.078528
2,1Q11,3,11.170565
3,1Q11,4,10.772341
4,1Q11,5,10.793099


In [8]:
predictions.head()

,Receptor,PoseRank,Predicted_Binding_Affinity
0,1Q11,1,4.336701
1,1Q11,2,4.221958
2,1Q11,3,4.097513
3,1Q11,4,4.077734
4,1Q11,5,4.081782


Now the frames can be easily combined into a single frame.

In [9]:
combined = docking.merge(predictions, sort=True)
combined[combined.PoseRank == 1]

,Receptor,PoseRank,RMSD,Predicted_Binding_Affinity
0,1Q11,1,10.889195,4.336701
7,1QY2,1,4.055790,5.032730
8,1TUV,1,3.461240,4.354066
12,1UKA,1,1.433749,4.241578
21,1Y2C,1,6.506677,5.494447
...,...,...,...,...
264,5E9I,1,3.670195,4.000142
268,5EP3,1,5.675027,4.820225
277,5F25,1,2.527784,5.033033
281,5FNQ,1,7.503415,4.485082


In [10]:
combined['RMSDRank'] = combined[['Receptor', 'RMSD']].groupby(by='Receptor').rank(ascending=True)
stats = combined.groupby(by='Receptor').min().rename(columns={'RMSD': 'MinRMSD'})
stats = stats.reset_index()[['Receptor', 'MinRMSD']]
stats = stats.merge(combined[combined.PoseRank == 1].rename(columns={'RMSD': 'TopPoseRMSD'})[['Receptor', 'TopPoseRMSD']])

## Did rescoring improve individual dockings?
Rescoring means assigning a new ranking order to the poses, in this case based on the predicted binding affinity. The higher the affinity, the better. We use RMSD to break ties in the affinity rank (lower RMSD means better rank).

In [11]:
affinity_rank = combined.sort_values(by='RMSD').groupby(by='Receptor').rank(ascending=False, method='first').rename(columns={'Predicted_Binding_Affinity': 'AffinityRank'}).AffinityRank
combined = combined.merge(affinity_rank, left_index=True, right_index=True, sort=True)
combined

,Receptor,PoseRank,RMSD,Predicted_Binding_Affinity,RMSDRank,AffinityRank
0,1Q11,1,10.889195,4.336701,3.0,1.0
1,1Q11,2,11.078528,4.221958,5.0,3.0
2,1Q11,3,11.170565,4.097513,6.0,5.0
3,1Q11,4,10.772341,4.077734,1.0,7.0
4,1Q11,5,10.793099,4.081782,2.0,6.0
...,...,...,...,...,...,...
284,5ILW,1,3.631839,4.862881,3.0,3.0
285,5ILW,2,0.587167,5.091005,1.0,1.0
286,5ILW,3,3.868636,4.811201,5.0,4.0
287,5ILW,4,3.838628,4.795057,4.0,5.0


In [12]:
stats = stats.merge(predictions.groupby(by='Receptor').max().Predicted_Binding_Affinity.rename('Max_Pose_Affinity'), left_on='Receptor', right_index=True)
stats = stats.merge(combined[combined.AffinityRank == 1][['Receptor', 'RMSD']].rename(columns={'RMSD': 'TopAffinityRMSD'}))
stats['DeltaRMSD'] = stats.TopPoseRMSD - stats.TopAffinityRMSD

To compare the old and new rankings for individual dockings, look at the RMSD difference between old and new top pose. When substracting the new from the old top pose RMSD, a difference > 0 shows an improvement after rescoring. For this analysis, we only use dockings with potential for improvement, i.e. where top pose RMSD is not the lowest RMSD. Below of these dockings all with a difference > 0 are shown. 

In [13]:
improvable_dockings = stats[stats.TopPoseRMSD != stats.MinRMSD][['Receptor', 'TopPoseRMSD', 'TopAffinityRMSD', 'DeltaRMSD']]
improved_dockings = improvable_dockings[improvable_dockings.DeltaRMSD > 0]
worse_dockings = improvable_dockings[improvable_dockings.DeltaRMSD < 0]
improved_dockings.sort_values(by='DeltaRMSD', ascending=False, ignore_index=True)

,Receptor,TopPoseRMSD,TopAffinityRMSD,DeltaRMSD
0,4CCW,12.245453,4.234023,8.011430
1,1Y2C,6.506677,1.243804,5.262873
2,5ILW,3.631839,0.587167,3.044672
3,3E62,6.774414,3.766407,3.008008
4,3P1F,5.471385,2.672168,2.799217
5,3UZJ,8.574695,5.953879,2.620817
6,5F25,2.527784,0.694955,1.832829
7,4G46,5.225079,3.784364,1.440715
8,5EP3,5.675027,4.711764,0.963263
9,4I6Q,1.162816,0.317071,0.845746


In [14]:
print(f'There are {len(improvable_dockings)} dockings with potential for improvement.')
print(round(len(improved_dockings)/len(improvable_dockings) * 100, 1), '% of which improved.')
print(round(len(worse_dockings)/len(improvable_dockings) * 100, 1), '% of which degraded.')
print('Improvable dockings:')
df_to_latex(stats[stats.TopPoseRMSD != stats.MinRMSD][['Receptor', 'TopPoseRMSD', 'MinRMSD']].sort_values(by='MinRMSD', ignore_index=True).round(4), ncol=3)

There are 41 dockings with potential for improvement.
48.8 % of which improved.
31.7 % of which degraded.
Improvable dockings:
3GKZ	&	1.7152	&	0.2263	&	3BU1	&	1.4928	&	0.7758	&	3E62	&	6.7744	&	3.7664 \\
1Y2C	&	6.5067	&	0.2569	&	4LVB	&	3.4452	&	1.0781	&	4G46	&	5.2251	&	3.7844 \\
4I6Q	&	1.1628	&	0.3171	&	4G28	&	1.7769	&	1.1672	&	4K5Z	&	4.1454	&	3.8047 \\
4E70	&	0.7532	&	0.3561	&	4D6S	&	2.0009	&	1.1769	&	3R4M	&	4.1121	&	4.0267 \\
3P1F	&	5.4714	&	0.4594	&	3VGN	&	1.224	&	1.182	&	4AHU	&	4.2981	&	4.1404 \\
4UVO	&	0.569	&	0.5039	&	4YAB	&	6.7273	&	1.3067	&	4CCW	&	12.2455	&	4.2032 \\
4ZSG	&	1.2048	&	0.5064	&	4NQ8	&	1.8485	&	1.5648	&	4Y4E	&	4.8023	&	4.6295 \\
4MNI	&	1.0244	&	0.5154	&	5E9I	&	3.6702	&	2.5026	&	4WY1	&	4.9598	&	4.8571 \\
4WMW	&	3.0335	&	0.5776	&	1TUV	&	3.4612	&	2.6653	&	3CHC	&	6.934	&	5.1189 \\
4P7X	&	2.1903	&	0.5819	&	3L4L	&	3.1965	&	2.8079	&	3UZJ	&	8.5747	&	5.9539 \\
5ILW	&	3.6318	&	0.5872	&	3PXK	&	3.0385	&	3.0358	&	4H9G	&	6.4585	&	6.2359 \\
4I6B	&	0.7371	&	0.6272	&	3MZ9	&	7.8806	&

## The big picture: Did rescoring improve overall docking results?
To further compare the rankings, count top poses with an RMSD in several intervals. This is based on the premise, that the docking program is able to produce good poses, but the scoring function fails to rank them first. If rescoring would do better than the prior used scoring function, the amount of poses with low RMSD would increase after rescoring.

In [15]:
top_docking_poses = docking[docking.PoseRank == 1]
n_dockings = len(top_docking_poses)
top_rescoring_poses = combined[combined.AffinityRank == 1]
thresholds = [0.5, 1.0, 1.5, 2.0, 2.5]
counts_docking = []
counts_rescoring = []
for threshold in thresholds:
    counts_docking.append(top_docking_poses[top_docking_poses.RMSD >= threshold - 0.5].loc[top_docking_poses.RMSD < threshold].count()[0])
    counts_rescoring.append(top_rescoring_poses[top_rescoring_poses.RMSD >= threshold - 0.5].loc[top_rescoring_poses.RMSD < threshold].count()[0])

pd.DataFrame({'RMSDThreshold': thresholds, 'TopPoseCountDocking': counts_docking, 'TopPoseCountRescoring': counts_rescoring})

,RMSDThreshold,TopPoseCountDocking,TopPoseCountRescoring
0,0.5,10,9
1,1.0,13,15
2,1.5,13,12
3,2.0,5,2
4,2.5,3,3


In [16]:
print(f'Before rescoring, {round(sum(counts_docking)/n_dockings * 100, 1)} % of dockings have a top pose with RMSD<2.5 A.')
print(f'After rescoring, {round(sum(counts_rescoring)/n_dockings * 100, 1)} % of dockings have a top pose with RMSD<2.5 A.')

Before rescoring, 53.7 % of dockings have a top pose with RMSD<2.5 A.
After rescoring, 50.0 % of dockings have a top pose with RMSD<2.5 A.


## A closer look: Ranking of pose with lowest RMSD
This is an analysis of the entire docking pipeline. Here we count what could be called correctly reproduced binding modes: the best pose with an RMSD<2.5 A ranked first.

In [17]:
top_docking_poses = combined[combined.PoseRank == 1].loc[combined.RMSDRank == 1]
top_rescoring_poses = combined[combined.AffinityRank == 1].loc[combined.RMSDRank == 1]

thresholds = [0.5, 1.0, 1.5, 2.0, 2.5]
counts_docking = []
counts_rescoring = []
for threshold in thresholds:
    counts_docking.append(top_docking_poses[top_docking_poses.RMSD >= threshold - 0.5].loc[top_docking_poses.RMSD < threshold].count()[0])
    counts_rescoring.append(top_rescoring_poses[top_rescoring_poses.RMSD >= threshold - 0.5].loc[top_rescoring_poses.RMSD < threshold].count()[0])

pd.DataFrame({'RMSDThreshold': thresholds, 'LowestRMSDTopPoseCountDocking': counts_docking, 'LowestRMSDTopPoseCountRescoring': counts_rescoring})

,RMSDThreshold,LowestRMSDTopPoseCountDocking,LowestRMSDTopPoseCountRescoring
0,0.5,10,9
1,1.0,10,13
2,1.5,7,8
3,2.0,2,1
4,2.5,1,1


In [18]:
print(f'Before rescoring, {round(sum(counts_docking)/n_dockings * 100, 1)} % of dockings correctly reproduced the binding mode.')
print(f'After rescoring, {round(sum(counts_rescoring)/n_dockings * 100, 1)} % of dockings correctly reproduced the binding mode.')

Before rescoring, 36.6 % of dockings correctly reproduced the binding mode.
After rescoring, 39.0 % of dockings correctly reproduced the binding mode.


## Validate prediction with reference ligands
By including the reference ligand into the affinity ranking, we can validate the affinity prediction. If it works well, the reference ligand should rank first. To make the validation a little less strict, exclude those dockings, where the top pose (affinity-wise) already has an RMSD<1 A. Below all reference ligands with a higher predicted affinity than the top affinity pose are shown.

In [19]:
refligs.rename(columns={'Predicted_Binding_Affinity': 'Reflig_Binding_Affinity'}, inplace=True)
refligs = refligs.merge(stats[stats.TopAffinityRMSD >= 1])
n_refligs_should_rank_first = len(refligs)
top_ranked_refligs = refligs.loc[refligs.Reflig_Binding_Affinity > refligs.Max_Pose_Affinity].reset_index(drop=True)
top_ranked_refligs[['Receptor', 'Reflig_Binding_Affinity', 'Max_Pose_Affinity']]

,Receptor,Reflig_Binding_Affinity,Max_Pose_Affinity
0,1Q11,4.542346,4.336701
1,1QY2,5.090575,5.032730
2,1TUV,4.484904,4.383816
3,1UKA,4.242390,4.241578
4,2CFI,5.755358,5.705528
5,2FP2,5.101708,5.074584
6,2RC9,5.168026,4.251784
7,2W6P,5.897585,5.695597
8,2YK1,6.084021,6.047810
9,3E62,4.696739,4.608440


In [20]:
print(n_refligs_should_rank_first, 'dockings have a top affinity pose with an RMSD>=1 A.')
print(f'{round(len(top_ranked_refligs)/n_refligs_should_rank_first * 100, 1)} % of reference ligands have a higher predicted binding affinity than all corresponding docking poses.')

58 dockings have a top affinity pose with an RMSD>=1 A.
43.1 % of reference ligands have a higher predicted binding affinity than all corresponding docking poses.


## Validate prediction with correlation of RMSD and affinity

Another metric of model performance is correlation between RMSD and predicted affinity. Here we calculate Pearson's correlation coefficient across all dockings, including reference ligands. Since affinity should increase when RMSD decreases, -1 means perfect correlation.

In [21]:
from scipy.stats import pearsonr

refligs['RMSD'] = 0
all_structures = pd.concat([combined[['Receptor', 'RMSD', 'Predicted_Binding_Affinity']], refligs.rename(columns={'Reflig_Binding_Affinity': 'Predicted_Binding_Affinity'})[['Receptor', 'RMSD', 'Predicted_Binding_Affinity']]])
p = pearsonr(all_structures.RMSD, all_structures.Predicted_Binding_Affinity)
print(f'Pearson\'s correlation coefficient is {round(p[0], 3)} with a p-value of {round(p[1], 3)}.')

Pearson's correlation coefficient is -0.163 with a p-value of 0.002.


Now, let's look at correlation for individual dockings. Note, that dockings can have very few poses, which leads to very high or very low correlation. To compensate for this, compute an average weighted by number of poses in each docking.

In [22]:
groups = all_structures.groupby(by='Receptor')
ids = []
ps = []
n_structs = []
for group in groups:
    if len(group[1]) > 1:
        p = pearsonr(group[1].RMSD, group[1].Predicted_Binding_Affinity)[0]
        ps.append(p)
        ids.append(group[0])
        n_structs.append(len(group[1]))

pearson = pd.DataFrame({'Receptor': ids, 'Pearson_R': ps, 'N_Structs': n_structs})
avg = pearson.Pearson_R.sum()/len(pearson)
weighted_avg = (pearson.Pearson_R * pearson.N_Structs).sum()/pearson.N_Structs.sum()
print(f'Average correlation across all dockings is {round(avg, 3)}.')
print(f'Average correlation weighted by number of poses is {round(weighted_avg, 3)}.')

Average correlation across all dockings is -0.284.
Average correlation weighted by number of poses is -0.241.


In [23]:
# Save (append) correlation to CSV file

import os

if os.path.exists('correlation.csv'):
    mode = 'a'
    header = False
else:
    mode = 'w'
    header = True
#pd.DataFrame({'ID': f'{model.upper()}-{ecif_cutoff}', 'Avg_Pearson_R': avg, 'Weighted_Avg_Pearson_R': weighted_avg}, index=[1]).to_csv('correlation.csv', index=False, mode=mode, header=header)